## Part III - Data Preprocessing

University of San Diego - MS Applied AI

AAI-540 Team 5

October 21, 2024

#### Set up

In [1]:
# setup environment
%run 0-Environment_Setup.ipynb

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Stored 's3_datalake_path_csv' (str)
Stored 'local_data_path_csv' (str)
Stored 's3_datalake_path_parquet' (str)


In [2]:
# check database name
print(database_name)

aai540finalprojectdb


In [5]:
# Create the database
wr.catalog.create_database(database_name)

In [35]:
# Load data
holidays_df = wr.s3.read_csv(s3_datalake_path_csv + "/holidays_events.csv")
oil_df = wr.s3.read_csv(s3_datalake_path_csv + "/oil.csv")
transactions_df = wr.s3.read_csv(s3_datalake_path_csv + "/transactions.csv")
stores_df = wr.s3.read_csv(s3_datalake_path_csv + "/stores.csv")
sales_df = wr.s3.read_csv(s3_datalake_path_csv + "/train.csv")

#### Clean Holidays and save to Parquet

In [7]:
# Check that the data loaded correctly
display(holidays_df.head())
holidays_df.info()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [8]:
# Convert dates to datetime (excluding timestamp)
holidays_df['date'] = pd.to_datetime(holidays_df['date']).dt.date

In [9]:
# Write cleaned data to Parquet
wr.s3.to_parquet(
    holidays_df, 
    path=f"{s3_datalake_path_parquet}/holidays", 
    dataset=True,
    database=database_name,
    table="holidays",
    filename_prefix="holidays"
)

{'paths': ['s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/holidays/holidays0b545ea75d9e431983a73b4b33cf7f5e.snappy.parquet'],
 'partitions_values': {}}

In [10]:
# Check that the table was created
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1566/3797932140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,holidays


In [11]:
# Check that data from the table loads correctly
table_name = "holidays"

# Define SQL query
statement = """SELECT * FROM {}.{}
    LIMIT 5""".format(
    database_name, table_name
)

# Display the results
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_1566/1244320953.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


#### Clean Oil and save to Parquet

In [12]:
# Check that the data loaded correctly
display(oil_df.head())
oil_df.info()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [13]:
# Convert dates to datetime (excluding timestamp)
oil_df['date'] = pd.to_datetime(oil_df['date']).dt.date

In [14]:
# Impute the missing values for oil prices by backfilling them
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()
oil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [15]:
# Write cleaned data to Parquet
wr.s3.to_parquet(
    oil_df, 
    path=f"{s3_datalake_path_parquet}/oil", 
    dataset=True,
    database=database_name,
    table="oil",
    filename_prefix="oil"
)

{'paths': ['s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/oil/oild42005058d584bfba73106d7a5029975.snappy.parquet'],
 'partitions_values': {}}

In [16]:
# Check that the table was created
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1566/3797932140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,holidays
1,oil


In [17]:
# Check that data from the table loads correctly
table_name = "oil"

# Define SQL query
statement = """SELECT * FROM {}.{}
    LIMIT 5""".format(
    database_name, table_name
)

# Display the results
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_1566/3206027839.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


#### Clean Transactions and save to Parquet

In [18]:
# Check that the data loaded correctly
display(transactions_df.head())
transactions_df.info()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [19]:
# Convert dates to datetime (excluding timestamp)
transactions_df['date'] = pd.to_datetime(transactions_df['date']).dt.date

In [20]:
# Write cleaned data to Parquet
wr.s3.to_parquet(
    transactions_df, 
    path=f"{s3_datalake_path_parquet}/transactions", 
    dataset=True,
    database=database_name,
    table="transactions",
    filename_prefix="transactions"
)

{'paths': ['s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/transactions/transactions0088811f08de4be8979875e3e7017d9f.snappy.parquet'],
 'partitions_values': {}}

In [21]:
# Check that the table was created
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1566/3797932140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,holidays
1,oil
2,transactions


In [22]:
# Check that data from the table loads correctly
table_name = "transactions"

# Define SQL query
statement = """SELECT * FROM {}.{}
    LIMIT 5""".format(
    database_name, table_name
)

# Display the results
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_1566/1481457548.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


#### Clean Stores and save to Parquet

In [23]:
# Check that the data loaded correctly
display(stores_df.head())
stores_df.info()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [24]:
# Write cleaned data to Parquet
wr.s3.to_parquet(
    stores_df, 
    path=f"{s3_datalake_path_parquet}/stores", 
    dataset=True,
    database=database_name,
    table="stores",
    filename_prefix="stores",
)

{'paths': ['s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/stores/stores189c91e1f4e64a0d9ca56a0d5bc0bd94.snappy.parquet'],
 'partitions_values': {}}

In [25]:
# Check that the table was created
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1566/3797932140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,holidays
1,oil
2,stores
3,transactions


In [26]:
# Check that data from the table loads correctly
table_name = "stores"

# Define SQL query
statement = """SELECT * FROM {}.{}
    LIMIT 5""".format(
    database_name, table_name
)

# Display the results
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_1566/3250605361.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Clean Sales and save to Parquet

In [36]:
# Check that the data loaded correctly
display(sales_df.head())
sales_df.info()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [37]:
# Convert dates to datetime
sales_df['date'] = pd.to_datetime(sales_df['date'])

# Extract year for partitioning
sales_df['year'] = sales_df['date'].dt.year

# Convert datetime to date (excluding timestamp)
sales_df['date'] = pd.to_datetime(sales_df['date']).dt.date

In [38]:
# Check for missing values
sales_df.isna().any()

id             False
date           False
store_nbr      False
family         False
sales          False
onpromotion    False
year           False
dtype: bool

In [39]:
# Drop the id column
sales_df.drop(columns=['id'], inplace=True)

In [40]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   store_nbr    int64  
 2   family       object 
 3   sales        float64
 4   onpromotion  int64  
 5   year         int32  
dtypes: float64(1), int32(1), int64(2), object(2)
memory usage: 125.9+ MB


In [41]:
# Write cleaned data to Parquet
wr.s3.to_parquet(
    sales_df, 
    path=f"{s3_datalake_path_parquet}/sales", 
    dataset=True,
    database=database_name,
    table="sales",
    partition_cols=["year"],
    filename_prefix="sales"
)

{'paths': ['s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2013/salesaf96531a4a614d56b2146a02efc97c0e.snappy.parquet',
  's3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2014/salesaf96531a4a614d56b2146a02efc97c0e.snappy.parquet',
  's3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2015/salesaf96531a4a614d56b2146a02efc97c0e.snappy.parquet',
  's3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2016/salesaf96531a4a614d56b2146a02efc97c0e.snappy.parquet',
  's3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2017/salesaf96531a4a614d56b2146a02efc97c0e.snappy.parquet'],
 'partitions_values': {'s3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2013/': ['2013'],
  's3://sagemaker-us-east-1-343218227212/store-sales-forecasting/parquet/sales/year=2014/': ['2014'],
  's3://sagemaker-us-east-1-343218227212/stor

In [42]:
# Check that the table was created
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1566/3797932140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,holidays
1,oil
2,sales
3,stores
4,transactions


In [43]:
# Check that data from the table loads correctly
table_name = "sales"

# Define SQL query
statement = """SELECT * FROM {}.{}
    LIMIT 5""".format(
    database_name, table_name
)

# Display the results
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_1566/3860274799.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,date,store_nbr,family,sales,onpromotion,year
0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013
1,2013-01-01,1,BABY CARE,0.0,0,2013
2,2013-01-01,1,BEAUTY,0.0,0,2013
3,2013-01-01,1,BEVERAGES,0.0,0,2013
4,2013-01-01,1,BOOKS,0.0,0,2013
